# Visualize a text from the Patristic Text Archive on a map
Annette von Stockhausen (CC-BY)

In [1]:
import os, json, jsonlines, gzip
import requests
import lxml
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster

### Exctract places from file

In [2]:
# load file
url = ("https://raw.githubusercontent.com/PatristicTextArchive/pta_data/9d05b6882ed19fc620f42599d13b6cd2bbff423c/data/pta0022/pta010/pta0022.pta010.pta-grc1.xml")
source = requests.get(url)

In [3]:
soup = BeautifulSoup(source.text, 'xml')
strip_tags = ['cit', 'ref', 'quote', 'said', 'gap'] # remove not needed tags to avoid problems
for tag in strip_tags: 
    for match in soup.find_all(tag):
        match.replaceWithChildren()
body = soup.find("text")
title = soup.find('title')
places = []
for place in body.find_all('placeName'):
    place_entry = {}
    refs = place.get("ref")
    place_name = place.text
    p = place.parent
    div = p.parent
    chapter = div["n"] 
    ct_before = place.previous_siblings
    context_before_list = []
    for context_b in ct_before:
        context_before_list.append(context_b.string)
    context_before = " ".join(context_before_list[::-1]).split()[-5:]
    ct_after = place.next_siblings
    context_after_list = []
    for context_a in ct_after:
        context_after_list.append(context_a.string)
    try:
        context_after = " ".join(context_after_list).split()[:5]
    except:
        context_after = ""
    context = chapter+" "+" ".join(context_before)+" <b>"+place_name+"</b> "+" ".join(context_after)
    if not any(d['ID'] == refs for d in places):
        place_entry["ID"] = refs
        place_entry["Context"] = context
        place_entry["Count"] = 1
        places.append(place_entry)
    else:
        matches = next(d for d in places if refs == d["ID"])
        tc = matches["Count"]
        myid = matches["ID"]
        mycontext = matches["Context"]
        place_entry["ID"] = myid
        place_entry["Context"] = mycontext+"<br/>"+context
        place_entry["Count"] = tc+1
        places.remove(matches)
        places.append(place_entry)
places_count = [d['Count'] for d in places]
places_count = sum(places_count)
print(title.text+" enthält "+str(places_count)+" Ortsnennungen und "+str(len(places))+" Orte.")
print(places)

Apologia ad Constantium enthält 56 Ortsnennungen und 26 Orte.
[{'ID': 'https://pleiades.stoa.org/places/197446', 'Context': '3 περὶ Θάλασσον ἦλθον εἰς τὴν <b>Πιτυβίωνα</b> καὶ ἡμεῖς ἐν τῇ Ἀκυληίᾳ', 'Count': 1}, {'ID': 'https://pleiades.stoa.org/places/393473', 'Context': '6 εἰπεῖν καὶ Κρισπῖνος ὁ τῆς <b>Πατάβων</b> καὶ Λούκιλλος ὁ ἐν Βερωνὶ', 'Count': 1}, {'ID': 'https://pleiades.stoa.org/places/383816', 'Context': '6 Πατάβων καὶ Λούκιλλος ὁ ἐν <b>Βερωνὶ</b> καὶ Διονύσιος ὁ ἐν Ἤλιδι', 'Count': 1}, {'ID': 'https://pleiades.stoa.org/places/570220', 'Context': '6 Βερωνὶ καὶ Διονύσιος ὁ ἐν <b>Ἤλιδι</b> καὶ Βικέντιος ὁ ἐν Καμπανίᾳ', 'Count': 1}, {'ID': 'https://pleiades.stoa.org/places/383706', 'Context': '6 Τριβέρεως καὶ Προτάσιος ὁ τῆς <b>Μεδιολάνου</b> , δύναται καὶ Εὐγένιος ὁ<br/>3 αὐτόν· ἦν δὲ ἐν τῇ <b>Μεδιολάνῳ</b> . ἐγὼ δὲ διερωτῶν τὴν<br/>4 κατελθὼν τοίνυν εἰς τὴν <b>Μεδιόλανον</b> εἶδον πολλὴν φιλανθρωπίαν· κατηξίωσε γὰρ', 'Count': 3}, {'ID': 'https://pleiades.stoa.org/places/20743

### Show on map

In [4]:
def load_pleiades_data():
    '''Load Pleiades data from http://atlantides.org/downloads/pleiades/json/pleiades-places-latest.json.gz if it does not exist yet'''
    if os.path.isfile('pleiades-places-latest.jsonl'):
        contents = open('pleiades-places-latest.jsonl', "r").read() 
    else:
        url = 'http://atlantides.org/downloads/pleiades/json/pleiades-places-latest.json.gz'
        r = requests.get(url, allow_redirects=True)
        with open('pleiades-places-latest.json.gz', 'wb') as f_gzip:
            f_gzip.write(r.content)
        with gzip.open("pleiades-places-latest.json.gz", 'rb') as f_in:
            with jsonlines.open("pleiades-places-latest.jsonl", mode='w', sort_keys=True,compact=True) as f_out:
                places = json.load(f_in)
                for place in places['@graph']:
                    f_out.write(place)
        contents = open('pleiades-places-latest.jsonl', "r").read() 
    data = [json.loads(str(item)) for item in contents.strip().split('\n')]
    return data

In [5]:
data = load_pleiades_data()

In [6]:
# base map
m = folium.Map(location=[40.4285312, 29.715356500000002],
               zoom_start=4,
              tiles = 'https://dh.gu.se/tiles/imperium/{z}/{x}/{y}.png',
              attr = '<a href="http://dh.gu.se/dare">DARE 2015</a> (CC-BY-SA), mapped data from <a href="http://pta.bbaw.de">Patristic Text Archive</a>',
              control_scale=True)

In [7]:
map_data = []
for place in range(len(places)):
    entries = []
    for key in places[place]:
        entries.append(places[place][key])
    entries = (entries)
    map_data.append(entries)
for place in map_data:
    location = next(item for item in data if item["uri"] == place[0])
    lc = location.get("reprPoint")
    try:
        lat = lc[1]
        long = lc[0]
    except:
        print(place[0]+" has no coordinates and is therefore not mapped.")
        pass      
    place_name = location.get("title")
    popup_entry = folium.Popup(place[1],
                     min_width=500,
                     max_width=500)
    mult = place[2]
    # Add marker to map
    folium.CircleMarker(
    radius=2*mult,
    location=[lat,long],
    tooltip=place_name,
    popup=popup_entry,
    color='crimson',
    fill=True,
    ).add_to(m)
    #folium.Marker([lat,long]).add_to(m)

https://pleiades.stoa.org/places/766 has no coordinates and is therefore not mapped.
https://pleiades.stoa.org/places/991375 has no coordinates and is therefore not mapped.


In [8]:
# show map
print("Places mentioned in "+title.text)
m

Places mentioned in Apologia ad Constantium


In [9]:

m.save('ApolConst-map.html')